<a href="https://colab.research.google.com/github/shubhanshukaintura/Credit-card-fraud-detection/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing dependencies

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

loading dataset to pandas data frame

In [3]:
credit_card_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/creditcard.csv')

In [4]:
#first five rows of dataframe
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
#dataframe information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
#checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [10]:
#distribution of legit transcation & fraud transaction data
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

this dataset is higly unbalanced as legit transaction are more compare to fraud transaction

0-> legit transaction
1-> fraud transcation

In [12]:
#separating the data for analysis
legit = credit_card_data[credit_card_data.Class==0]
fraud = credit_card_data[credit_card_data.Class==1]

In [13]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [14]:
#statistacal measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [15]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [16]:
#compare the values for both transcation
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


**Under-sampling**

Build a sample dataset containing similar distribution of normal transcations and fraudent transcations

number of fraudent transcation is -> 492

In [17]:
legit_sample= legit.sample(n=492)

conacatinating two data frame

In [19]:
new_dataset = pd.concat([legit_sample,fraud],axis=0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
262138,160333.0,-0.511518,0.947141,1.186131,-0.403563,0.554017,-0.264703,0.962014,-0.010264,-0.852164,...,-0.256087,-0.765858,-0.331691,-0.336561,0.375177,0.463459,-0.027730,0.035078,8.13,0
141534,84379.0,-0.862899,0.343549,1.989119,0.274251,0.699374,1.317684,0.119821,0.172092,0.260285,...,0.019495,0.444679,-0.099875,-1.132573,-0.186974,-0.450045,-0.228641,-0.036222,9.99,0
190834,128999.0,-1.009132,1.795755,-1.939447,-1.375346,0.925896,-0.843745,0.818783,0.606381,-0.683893,...,0.326343,0.822401,-0.166974,0.234427,-0.170032,0.064800,0.103136,0.192352,3.70,0
67283,52453.0,-0.354183,1.018814,1.267704,0.078608,-0.063410,-1.027601,0.604837,0.003448,-0.194695,...,-0.278152,-0.798315,0.001282,0.292992,-0.203767,0.076622,0.236786,0.095758,1.98,0
191902,129449.0,-2.327604,-2.210617,0.498796,0.129895,3.163913,-3.666150,-0.502482,-0.270108,-1.923320,...,0.164142,0.079151,0.379700,0.820738,0.245315,-0.677180,0.033313,0.221135,14.35,0


In [20]:
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [21]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,93594.880081,0.013875,-0.070217,0.020210,-0.041872,0.086066,0.102920,-0.025674,0.078178,0.021243,...,-0.013053,-0.025123,-0.035905,0.009905,0.023043,0.023086,0.031504,-0.010788,-0.003756,76.782134
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


splitting the dataset into features & targets

In [23]:
X = new_dataset.drop(columns='Class',axis=1)
Y = new_dataset['Class']
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
262138  160333.0 -0.511518  0.947141  1.186131 -0.403563  0.554017 -0.264703   
141534   84379.0 -0.862899  0.343549  1.989119  0.274251  0.699374  1.317684   
190834  128999.0 -1.009132  1.795755 -1.939447 -1.375346  0.925896 -0.843745   
67283    52453.0 -0.354183  1.018814  1.267704  0.078608 -0.063410 -1.027601   
191902  129449.0 -2.327604 -2.210617  0.498796  0.129895  3.163913 -3.666150   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [24]:
print(Y)

262138    0
141534    0
190834    0
67283     0
191902    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


split the data into training and test data

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [28]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


**Model Training**

Logistic Regression

In [29]:
model = LogisticRegression()

training the model with training data

In [30]:
model.fit(X_train,Y_train)

LogisticRegression()

Model Evaluation

Accuracy score

In [32]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [33]:
print("Accuracy on training data : ",training_data_accuracy)

Accuracy on training data :  0.9224904701397713


In [34]:
#accuracy on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [35]:
print('Accuracy on testing data : ', testing_data_accuracy)

Accuracy on testing data :  0.9137055837563451
